In [99]:
import spotipy
import spotipy.oauth2 as oauth2
import pprint
import pandas as pd
import pickle
import numpy as np
import csv

In [76]:
#Read in file containing authentification info
with open('spotify_key.txt') as f:
    data = f.readlines()
    
client_id = data[0].split(':')[1].replace('\n', '')
client_secret = data[1].split(':')[1].replace('\n', '')

#Authorize credentials
credentials = oauth2.SpotifyClientCredentials(
        client_id=client_id,
        client_secret=client_secret)

token = credentials.get_access_token()
spotify = spotipy.Spotify(auth=token)

In [41]:
# Read in the master dataset
data = pd.read_csv('working_data.nosync/spotify_data.csv')

In [66]:
data.shape

(612405, 24)

In [91]:
#Function to extract whether the song is explicit
def is_explicit(spotify_ids):
    
    out={}
    
    counter=0
    
    #Initialize
    token = credentials.get_access_token()
    spotify = spotipy.Spotify(auth=token)
    
    for spotify_id in spotify_ids:
        #Get track info
        try:
            track = spotify.track(spotify_id)
            out[spotify_id] = track['explicit']
            counter+=1
            if counter % 1000 == 0:
                print('completed songs: ', counter)
                #save data in case of crash periodically
                with open('spotify_genres.nosync/explicit_update1.pkl', 'wb') as f:
                    pickle.dump(out, f) 

        except spotipy.client.SpotifyException:
            #reauthorize
            token = credentials.get_access_token()
            spotify = spotipy.Spotify(auth=token)
            counter-=1

        except ValueError:
            out[spotify_id] = 'not_found'
            
        
    return out

In [ ]:
#Had to rerun this part by part due to Spotify restrictions
explicit = is_explicit(data.SPOTIFY_ID[141000+63000+292000+11000+12000:])

In [45]:
with open('spotify_genres.nosync/explicit7.pkl', 'wb') as f:
                    pickle.dump(explicit, f)

In [56]:
! ls 'spotify_genres.nosync' 

1100.pkl      explicit2.pkl explicit4.pkl explicit6.pkl progress.pkl
explicit.pkl  explicit3.pkl explicit5.pkl explicit7.pkl progress2.pkl


In [57]:
explicit_files = ['explicit2.pkl', 'explicit4.pkl', 'explicit6.pkl', 'explicit3.pkl', 'explicit5.pkl',
                  'explicit7.pkl']

In [62]:
#Read in and concatenate data

merged = {}

for file in explicit_files:
    recorded = open("spotify_genres.nosync/"+file, 'rb')
    extracted = pickle.load(recorded)
    merged.update(extracted)

In [70]:
len(merged.keys())

471342

In [72]:
start_ids = data.SPOTIFY_ID

In [88]:
not_found = [x for x in start_ids if x not in merged.keys()]

In [92]:
#Had to rerun this part by part due to Spotify restrictions
explicit = is_explicit(not_found)

In [93]:
merged.update(explicit)

In [94]:
len(merged)

612405

In [120]:
#Save data into file
output_df = pd.DataFrame.from_dict(merged, orient='index')
output_df = output_df.reset_index()
output_df.columns = ['SPOTIFY_ID', 'Explicit']

output_df.to_csv('working_data.nosync/explicit_spotifyid.csv', index=False)

In [121]:
#Check output
check = pd.read_csv('working_data.nosync/explicit_spotifyid.csv')

/Users/ksenia/anaconda/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [122]:
check.head()

,SPOTIFY_ID,Explicit
0,0Z4CsIA2WIyLPuoKlSLBvW,False
1,4zEBMipqCInhD3xl9Pzcuo,False
2,3SH6v9OCnyJVywo14HqVR8,False
3,0PQMqjrVIH3f1bhILdiaRZ,False
4,1l9zh64umSqYms1JP5REpi,False
